In [ ]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
import os
from selenium.webdriver.common.action_chains import ActionChains
import time
import re
import getpass
import json
from graph_tool.all import *

def login():
	username = raw_input('username: ')
	password = getpass.getpass()
	chromeDriver = '/Users/Jai/Downloads/chromedriver'
	chrome_options = webdriver.ChromeOptions()
	prefs = {"profile.managed_default_content_settings.notifications": 2}
	chrome_options.add_experimental_option("prefs", prefs)

	os.environ["webdriver.chrome.driver"] = chromeDriver
	driver = webdriver.Chrome(chromeDriver, chrome_options=chrome_options)

	driver.get("https://web.facebook.com/")
	assert 'Facebook' in driver.title
	email_field = driver.find_element_by_id("email")
	pass_field = driver.find_element_by_id("pass")

	email_field.send_keys(username)
	pass_field.send_keys(password)
	pass_field.send_keys(Keys.RETURN)
	return driver

def getLikesFromPosts(target, driver):
	# open the persons profile
	pageURL = str()
	if re.match(r'https?\:\/\/www\.facebook\.com\/[^?]*\?',target):
		pageURL = target
	else:
		pageURL = "https://web.facebook.com/" + target
	driver.get(pageURL)
	# TODO: replace with code that ensures we are at the bottom of the page
	for i in range(2):
		for i in range(2):
			ActionChains(driver).key_down(Keys.DOWN).perform()
		time.sleep(0.1)
	coverPhotoObj = driver.find_element_by_xpath('//*[@id="fbCoverImageContainer"]')
	coverPhotoObj.click()
	time.sleep(5)
	friends = set()
	reactionPageURLs = set()
	CoverObjectReactionElements = set()
	allCoverLinks = driver.find_elements_by_tag_name('a')
	for x in allCoverLinks:
		if '/ufi/reaction/profile/browser/' in str(x.get_attribute('href')):
			CoverObjectReactionElements.add(x.get_attribute('href'))
	reactionPageURLs = reactionPageURLs.union(CoverObjectReactionElements)

	closeButtonObj = driver.find_element_by_xpath('//*[@id="photos_snowlift"]/div[2]/div/a')
	closeButtonObj.click()
	time.sleep(1)
	DPObj = driver.find_element_by_class_name('profilePic')
	DPObj.click()
	time.sleep(5)
	DPReactionElements = set()
	allDPLinks = driver.find_elements_by_tag_name('a')
	for x in allDPLinks:
		if '/ufi/reaction/profile/browser/' in str(x.get_attribute('href')):
			DPReactionElements.add(x.get_attribute('href'))
	reactionPageURLs = reactionPageURLs.union(DPReactionElements)

	# print "links found:"
	# for URL in reactionPageURLs:
	# 	print URL
	for element in reactionPageURLs:
		link = element
		print "exploring " + str(link)
		friendsFoundFromLink = getLikesFromReactionPage(link, driver)
		friends = friends.union(friendsFoundFromLink)
	return friends


def getLikesFromReactionPage(likelistURL, driver):
	driver.get(likelistURL)
	more = True
	while more:
		try:
			time.sleep(5)
			seemorebutton = driver.find_element_by_xpath('//*[@id="reaction_profile_pager1"]/div')
			seemorebutton.click()
		except NoSuchElementException:
			more = False
	friendlinks = driver.find_elements_by_xpath(
		'//*[@id="reaction_profile_browser1"]/li/div/div/div/div[2]/div[2]/div/a')
	result = set()
	for linkelement in friendlinks:
		# result.add(linkelement.get_attribute("href"))
		link = linkelement.get_attribute("href")
		name = linkelement.text
		idsearch = re.search(r'(https?\:\/\/(www\.)?(web\.)?facebook\.com\/)([^?]*)\?',name)
		if idsearch:
			print "adding " + idsearch.group(4)
			result.add((idsearch.group(4),name))
	return result

def genFriendGraph(root, friendList):
	'''
	To generate the graph for a given root node and his/her friend list
	:param root: the root node (as an fb-id)
	:param friendList: a list of tuples of (link,name)
	:return: the graph object
	'''
	g = Graph()
	root = g.add_vertex()
	g.gp.idtovert = g.new_gp("python::object")
	g.gp.idtovert = dict()
	g.vp.vertexText = g.new_vp("string")
	g.vp.id = g.new_vp("string")
	for friend in friendList:
		friendVertex = g.add_vertex()
		g.gp.idtovert[friend[0]] = g.vertex_index[friendVertex]
		g.vp.vertexText[friendVertex] = friend[1]
		print "making node with id " + friend[0]
		g.vp.id[friendVertex] = friend[0]
		g.add_edge(root,friendVertex)
	return g


def addNewFriends(g,target, friends,isfriend=True):
	'''
	:param g: the graph Im working with right now
	:param target: the person who I've clicked on - who's friends I'm adding to the graph
	:param friends: A list of (link,name) tuples of the friends of the person in consideration
	:return: the modified graph
	'''
	if not isinstance(g, Graph ):
		return False
	if isfriend:
		targetVertex = g.gp.idtovert[target]
	else:
		targetVertex = g.add_vertex()
	for friend in friends:
		if friend[0] not in g.gp.idtovert:
			friendVertex = g.add_vertex()
		else:
			friendVertex = g.vertex(g.gp.idtovert[friend[0]])
		g.gp.idtovert[friend[0]] = g.vertex_index[friendVertex]
		g.vp.id[friendVertex] = friend[0]
		g.vp.vertexText[friendVertex] = friend[1]
		print "adding edge to " + friend[1]
		g.add_edge(targetVertex,friendVertex)
	return g

In [ ]:
username = raw_input('username: ')
password = getpass.getpass()
driver = login(username, password)
partial_likes = getLikesFromPosts("sahilarora946", driver)
socialGraph = genFriendGraph("sahilarora946",partial_likes)
pos = radial_tree_layout(socialGraph,socialGraph.vertex(0))

In [ ]:
#pos = radial_tree_layout(socialGraph,socialGraph.vertex(0))
#graph_draw(socialGraph, pos = pos, vertex_text = socialGraph.vp.vertexText, vertex_size = 10, vertex_font_size = 10)
#values = socialGraph.list_properties()

In [ ]:
(newpos, selected ) = interactive_window(socialGraph, pos = pos, vertex_text = socialGraph.vp.vertexText, vertex_size = 10, vertex_font_size = 10)
changedvert = socialGraph.vp.id[socialGraph.vertex(0)]
for i in socialGraph.vertices():
    if selected[i]:
        changedvert=socialGraph.vp.id[i]
print "now exploring "+ changedvert
#get new friends, add to graph, and redraw the graph surface
newLikes = getLikesFromPosts(changedvert,driver)

In [ ]:
newids = set()
for like in newLikes:
    idsearch = re.search(r'(https?\:\/\/www\.facebook\.com\/)([^?]*)\?',str(like[0]))
    print "exploring " + like[1]
    if idsearch:
        newids.add( idsearch.group(2))
        print idsearch.group(2)


In [ ]:
socialGraph = addNewFriends(socialGraph,changedvert,newids)
pos = radial_tree_layout(socialGraph,socialGraph.vertex(0))
graph_draw(socialGraph, pos = pos, vertex_text = socialGraph.vp.vertexText, vertex_size = 10, vertex_font_size = 10)

In [ ]:
def openInteractiveWindow(g,driver):
    if not isinstance(g, Graph):
        return False
    graph_draw(g, vertex_text = socialGraph.vp.vertexText, vertex_size = 10, vertex_font_size = 10, inline=True)
    #get the id of the guy to explore now
    print " Input as follows \n 1 for importing a new person into the graph \n 2 for exploring an existing person in the graph \n 3 for exit"
    choice = input()
    if choice == 1:
        newid = raw_input("enter the ID of the person to explore")
        g = addNewFriends(g,newid, getLikesFromPosts(newid,driver),isfriend=False)
        openInteractiveWindow(g,driver)
    elif choice ==2:
        explorename = raw_input("enter the name of the person you want to explore")
        exploreids = [g.vp.id[x] for x in g.vertices() if g.vp.vertexText[x] == explorename]
        print "choose from one of the following:"
        print exploreids
        exploreid = raw_input()
        g = addNewFriends(g,exploreid,getLikesFromPosts(exploreid, driver))
        openInteractiveWindow(g,driver)
    else:
        return True
    '''
    (newpos, selected ) = interactive_window(g, vertex_text = socialGraph.vp.vertexText, vertex_size = 10, vertex_font_size = 10)
    changedvert = g.vp.id[g.vertex(0)]
    for i in g.vertices():
        if selected[i]:
            changedvert=g.vp.id[i]
    print "now exploring "+ changedvert
    #get new friends, add to graph, and redraw the graph surface
    newLikes = getLikesFromPosts(changedvert,driver)
    newids = set()
    for like in newLikes:
        idsearch = re.search(r'(https?\:\/\/www\.facebook\.com\/)([^?]*)\?',str(like[0]))
        if idsearch:
            newids.add( (idsearch.group(2),like[1]))
    g = addNewFriends(g,changedvert,newids)
    openInteractiveWindow(g)
    '''
    
def genGraphMultiUsers(idlist,driver):
    g = Graph()
    firstlikes = getLikesFromPosts(idlist[0],driver)
    firstids = set()
    for like in firstlikes:
        idsearch = re.search(r'(https?\:\/\/www\.facebook\.com\/)([^?]*)\?',str(like[0]))
        if idsearch:
            firstids.add((idsearch.group(2),like[1]))
    g = genFriendGraph(idlist[0],firstids)
    for user in idlist:
        if user == idlist[0]:
            continue
        newLikes = getLikesFromPosts(user,driver)
        newids = set()
        for like in newLikes:
            idsearch = re.search(r'(https?\:\/\/(www\.)?(web\.)?facebook\.com\/)([^?]*)\?\?',str(like[0]))
            if idsearch:
                newids.add( (idsearch.group(4),like[1]))
        g = addNewFriends(g,user,newids,isfriend=False)
    return g

In [ ]:
#driver = login()
socialGraph = genGraphMultiUsers(["happy.sachdeva","sahilarora946"],driver)

In [ ]:
#driver = login()
socialGraph = genFriendGraph('sahilarora946',getLikesFromPosts('sahilarora946',driver))

In [ ]:
print socialGraph.num_vertices()

In [ ]:
#driver = login()
openInteractiveWindow(socialGraph,driver)
#graph_draw(socialGraph, inline =True)

In [ ]:
from graph_tool.all import *
socialGraph= Graph()
socialGraph.load('/Users/Jai/Documents/work/shipsy/bitbucket/WVRN Algo/socialGraph.xml')

In [ ]:
socialGraph.clear_filters()
filt= socialGraph.new_vp("bool")
#socialGraph.clear_vertex(socialGraph.vertex(0))
socialGraph.clear_vertex(socialGraph.vertex(913))
socialGraph.set_vertex_filter(filt,inverted=True)

In [ ]:
for v in socialGraph.vertices():
    if v.out_degree()+v.in_degree()<5:
        filt[v]=True
socialGraph.set_vertex_filter(filt,inverted=True)

In [ ]:
positions,moved = graph_draw(socialGraph)

In [ ]:
spatial_map = list()
for v in socialGraph.vertices():
    spatial_map.append(tuple(positions[v]))

In [ ]:
for i in range(len())